In [1]:
import re
import math
import numpy as np
from pprint import pprint
from subprocess import getstatusoutput
from doped import analysis
from matplotlib import pyplot as plt

def get_free_energy(OUTCAR_file_path):
    input = "gawk '/TOTEN/' " + OUTCAR_file_path
    get_free_energy = getstatusoutput(input)
    if get_free_energy[0] == 0:
        free_energy_list = re.findall(r"=(.+?)eV", get_free_energy[1])
        try:
            free_energy_list = [float(free_energy.strip()) 
                                for free_energy in free_energy_list]
        except ValueError:
            free_energy_list = ['NaN']
        return free_energy_list
    else:
        return ['NaN']

def charge_correction(filepath, defect_directory, stastic_dielec):
    defect_entry = analysis.defect_entry_from_paths(defect_path=filepath+defect_directory, 
                                                    bulk_path=filepath+"pft",
                                                    dielectric=stastic_dielec)
    print(f"Charge: {defect_entry.charge_state} at site: {defect_entry.defect_supercell_site.frac_coords}")
    correction, error = defect_entry.get_kumagai_correction(return_correction_error=True)
    print(f"error range: {error}")
    correction = defect_entry.corrections['kumagai_charge_correction']
    return correction, error

def plot_formation_enthalpy_fermi_level_diagram(defect_data, band_structure):
    level_span = max(defect_data[3:6])+0.5
    enthalpy_span = np.linspace(defect_data[0]-0.5, defect_data[1]+0.5, 120)
    E_fermi = np.arange(0, level_span+0.5, 0.1)
    E_formation_Hip = E_fermi + defect_data[0]
    E_formation_Him = -E_fermi + defect_data[1]
    E_formation_Hin = 0 * E_fermi + defect_data[2]
    E_formation = [E_formation_Hip, E_formation_Him, E_formation_Hin]
    gap = np.ones(enthalpy_span.size)*band_structure[2]


filepath = "/mnt/d/Workspace/Data/ZrO2/defect/P21c/0GPa/VO/"
VBM = 5.6336
CBM = 9.2633
BAND_GAP = 3.6297
chemical_potential_Zr = -17.01668172 / 2
chemical_potential_O = -28.91053694 / 6
enthalpy_pft = get_free_energy(filepath+"pft/OUTCAR")[-1]
chemical_potential_O_poor = (enthalpy_pft / 32 - chemical_potential_Zr) / 2
chemical_potential_O_rich = chemical_potential_O
high_freq_dielec = np.array([[5.328996, 0.000000, 0.154152],
                            [0.000000, 5.362658, 0.000000],
                            [0.154152, 0.000000, 5.005340]])
ionic_dielec = np.array([[19.107809, 0.000000, 1.357205],
                        [0.000000, 17.240765, -0.000000],
                        [1.357205, -0.000000, 13.257194]])
stastic_dielec = high_freq_dielec + ionic_dielec
print(chemical_potential_O_poor, chemical_potential_O_rich)

-10.13849297828125 -4.818422823333333


In [2]:
enthalpy_VO_1 = get_free_energy(filepath+"VO_1/OUTCAR")[-1]
print(enthalpy_VO_1)
intercept_O_poor = enthalpy_VO_1 - enthalpy_pft + chemical_potential_O_poor 
intercept_O_rich = enthalpy_VO_1 - enthalpy_pft + chemical_potential_O_rich
print(intercept_O_poor, intercept_O_rich)

-910.07163187
0.9203332817186833 6.2404034366665995


In [3]:
enthalpy_VO1plus_1 = get_free_energy(filepath+"VO1+_1/OUTCAR")[-1]
print(enthalpy_VO1plus_1)
defect_entry = analysis.defect_entry_from_paths(defect_path=filepath+"VO1+_1/", bulk_path=filepath+"pft/", dielectric=stastic_dielec)
print(f"Charge: {defect_entry.charge_state} at site: {defect_entry.defect_supercell_site.frac_coords}")
correction, error = defect_entry.get_kumagai_correction(return_correction_error=True)
intercept_O_poor = enthalpy_VO1plus_1 - enthalpy_pft + chemical_potential_O_poor 
intercept_O_rich = enthalpy_VO1plus_1 - enthalpy_pft + chemical_potential_O_rich
print(intercept_O_poor, intercept_O_rich)

-918.73848174
Charge: 1 at site: [0.53278853 0.58633163 0.42458197]
Calculated Kumagai (eFNV) correction is 0.027 eV
-7.746516588281281 -2.426446433333365


In [4]:
enthalpy_VO2plus_1 = get_free_energy(filepath+"VO2+_1/OUTCAR")[-1]
print(enthalpy_VO1plus_1)
defect_entry = analysis.defect_entry_from_paths(defect_path=filepath+"VO2+_1/", bulk_path=filepath+"pft/", dielectric=stastic_dielec)
print(f"Charge: {defect_entry.charge_state} at site: {defect_entry.defect_supercell_site.frac_coords}")
correction, error = defect_entry.get_kumagai_correction(return_correction_error=True)
intercept_O_poor = enthalpy_VO2plus_1 - enthalpy_pft + chemical_potential_O_poor 
intercept_O_rich = enthalpy_VO2plus_1 - enthalpy_pft + chemical_potential_O_rich
print(intercept_O_poor, intercept_O_rich)

-918.73848174
Charge: 2 at site: [0.53278853 0.58633163 0.42458197]
Calculated Kumagai (eFNV) correction is 0.219 eV
-16.17819443828131 -10.858124283333392


In [5]:
enthalpy_VO_2 = get_free_energy(filepath+"VO_2/OUTCAR")[-1]
intercept_O_poor = enthalpy_VO_2 - enthalpy_pft + chemical_potential_O_poor 
intercept_O_rich = enthalpy_VO_2 - enthalpy_pft + chemical_potential_O_rich
print(intercept_O_poor, intercept_O_rich)

0.8020094717186907 6.122079626666607


In [6]:
enthalpy_VO1plus_2 = get_free_energy(filepath+"VO1+_2/OUTCAR")[-1]
print(enthalpy_VO1plus_1)
defect_entry = analysis.defect_entry_from_paths(defect_path=filepath+"VO1+_2/", bulk_path=filepath+"pft/", dielectric=stastic_dielec)
print(f"Charge: {defect_entry.charge_state} at site: {defect_entry.defect_supercell_site.frac_coords}")
correction, error = defect_entry.get_kumagai_correction(return_correction_error=True)
intercept_O_poor = enthalpy_VO1plus_2 - enthalpy_pft + chemical_potential_O_poor 
intercept_O_rich = enthalpy_VO1plus_2 - enthalpy_pft + chemical_potential_O_rich
print(intercept_O_poor, intercept_O_rich)

-918.73848174
Charge: 1 at site: [0.27462902 0.62157977 0.26201714]
Calculated Kumagai (eFNV) correction is 0.023 eV
-7.402700228281276 -2.08263007333336


In [7]:
enthalpy_VO2plus_2 = get_free_energy(filepath+"VO2+_2/OUTCAR")[-1]
print(enthalpy_VO1plus_1)
defect_entry = analysis.defect_entry_from_paths(defect_path=filepath+"VO2+_2/", bulk_path=filepath+"pft/", dielectric=stastic_dielec)
print(f"Charge: {defect_entry.charge_state} at site: {defect_entry.defect_supercell_site.frac_coords}")
correction, error = defect_entry.get_kumagai_correction(return_correction_error=True)
intercept_O_poor = enthalpy_VO2plus_2 - enthalpy_pft + chemical_potential_O_poor 
intercept_O_rich = enthalpy_VO2plus_2 - enthalpy_pft + chemical_potential_O_rich
print(intercept_O_poor, intercept_O_rich)

-918.73848174
Charge: 2 at site: [0.27462902 0.62157977 0.26201714]
Calculated Kumagai (eFNV) correction is 0.192 eV
-15.49997086828131 -10.179900713333392
